In [1]:
from __future__ import division, print_function
import numpy, pylab, math
import icecube
from icecube import dataio
from icecube import dataclasses
from icecube import CascadeVariables
import os
%matplotlib inline

In [6]:
#Set the GCD to use
gcd=dataio.I3File("/data/sim/IceCube/2011/filtered/level2/neutrino-generator/10099/00000-00999/GeoCalibDetectorStatus_IC86.55697_corrected_V2.i3.gz")


#Extract geometry object
gcd.rewind()
fr=gcd.pop_frame()
geo=fr.Get("I3Geometry")

In [8]:
#Load files
NFiles=1
sigFilesI3=[]
sigDir="/data/user/lwille/dat/2013NuTauComparison/"
sigFiles=os.listdir(sigDir)
for i in sigFiles:
    if(i.find(".i3.bz2")>0):
        sigFilesI3.append(i)

sigFilesI3=sigFilesI3[0:NFiles]

In [15]:
#Load the profiles
import cPickle
KernelsFile=open("DOMKernelsATWD.dat",'r')
KernelDictionary=cPickle.load(KernelsFile)



In [ ]:
#Extract the waveforms and distances
Count=0    
Distances=[]
Waveforms=[]
Energies=[]
Times=[]
Charges=[]
MaxWFTimes=[]
BinWidths=[]

EnergyLimLow=5e3
EnergyLimHigh=1e6

for sgFile in sigFilesI3[0:1]:
    Count=Count+1
    print("opening "+sgFile+",  "+str(Count)+" of " + str(len(sigFilesI3)))
    sigFile=dataio.I3File(sigDir+sgFile)
    sigFile.rewind()
    while(sigFile.more()):
        #pop a frame
        fr=sigFile.pop_physics()

        #grab the weight dict and event weight
        WeightDict=fr.Get("I3MCWeightDict")
        Weight=(365*24*3600)*1.0e-8*pow(WeightDict["PrimaryNeutrinoEnergy"], -2)*WeightDict["OneWeight"]/(WeightDict["NEvents"]*len(sigFilesI3))

        #Find the cascade center
        if not fr.Has("CascadeLlhVertexFit_L2") :
            continue

        if( WeightDict["PrimaryNeutrinoEnergy"]<EnergyLimLow) or (WeightDict["PrimaryNeutrinoEnergy"]>EnergyLimHigh):    
            continue
        csc=fr.Get('CascadeLlhVertexFit_L2')
        cscpos=csc.pos

        wf=fr.Get('CalibratedWaveformsHLCATWD')
        DistancesThisEvt=[]
        WaveformsThisEvt=[]
        EnergiesThisEvt=[]
        TimesThisEvt=[]
        ChargesThisEvt=[]
        BinWidthsThisEvt=[]
        for om, wf_series in wf:
            for w in wf_series:
                if(w.status==0):
                    DOMPos=geo.omgeo[om].position
                    DistancesThisEvt.append((cscpos-DOMPos).r)
                    WaveformsThisEvt.append(w.waveform)
                    TimesThisEvt.append(w.time)
                    ChargesThisEvt.append(sum(w.waveform))
                    BinWidthsThisEvt.append(w.bin_width)
                    break
        Energies.append(csc.energy)
        Distances.append(DistancesThisEvt)
        Waveforms.append(WaveformsThisEvt)
        Charges.append(ChargesThisEvt)
        Times.append(TimesThisEvt)
        BinWidths.append(BinWidthsThisEvt)
        MaxWFTimes.append(TimesThisEvt[numpy.argmax(ChargesThisEvt)])

    sigFile.close()

In [ ]:
#Extract the time shifts
TimeShifts=[]
for tindex in range(0,len(Times)):
    StartTime=numpy.min(Times[tindex])
    TimeShifts.append((Times[tindex]-StartTime)/BinWidths[tindex])
    

# Template Matching Method

In [ ]:
#This function applies the template matching
def MakeTemplateMatch(Waveform,Kernel):
    buffers=[]
    for istart in range(len(Waveform),0,-1):
        buff=numpy.zeros(istart)
        thebuffer=numpy.sum(numpy.concatenate((buff,Kernel))*numpy.concatenate((Waveform,buff)))
        buffers.append(thebuffer) 

    for istart in range(0,len(Waveform)):
        buff=numpy.zeros(istart)
        thebuffer=numpy.sum(numpy.concatenate((buff, Waveform))*numpy.concatenate((Kernel,buff)))
        buffers.append(thebuffer) 
    return buffers

In [ ]:
# Try it for a few waveforms 
DistanceLims=KernelDictionary.keys()
WFLength=1000
Offset=0
todo=10
pylab.figure(figsize=(5,5))

for evt in range(0,len(Waveforms[0:todo])):
    SumCorrel=numpy.zeros(2.*WFLength)
    for wf in range(0,len(Waveforms[evt])):
        Shift=int(TimeShifts[evt][wf])+Offset
        Kernel=None


        for dis in range(0,len(DistanceLims)-1):
            if(DistanceLims[dis]<Distances[evt][wf] and DistanceLims[dis+1]>Distances[evt][wf]) and len(Waveforms[evt][wf])==128:
                Kernel=WFDictionary[DistanceLims[dis]]
                break
        if(Kernel==None):
            continue
        if(Shift>WFLength):
            continue

        ToMatchWF=numpy.zeros(WFLength)

        for w in range(0,min(128,(WFLength-Shift))):
            ToMatchWF[w+Shift]+=numpy.array(Waveforms[evt][wf][w])
        SumCorrel+=MakeTemplateMatch(ToMatchWF,Kernel)
    print(evt)
    peak=numpy.argmax(SumCorrel)
    pylab.plot(range(0-peak,len(SumCorrel)-peak),SumCorrel/max(SumCorrel),label=str(int(Energies[evt]/1000)))
    pylab.legend(loc='upper left',title='E / TeV')
    pylab.title("Tau CC")
    pylab.xlim(-500,200)

    


# FFT Method

In [ ]:
MaxDist=150
todo=10
Count=0
PlotThem=True
AllSums=numpy.zeros(128,dtype='float')
for i in range(0, len(Energies)):
    N=0
    SumForm=numpy.zeros(128,dtype='float')
    FreqSum=numpy.zeros(65,dtype='complex')
    for j in range(0,len(Distances[i])):
        if Distances[i][j]<MaxDist:
            N+=1
            for distkey in KernelDictionary.keys():
                if Distances[i][j]>distkey:
                    break
            Kernel=KernelDictionary[distkey]
            Filter=numpy.abs(Kernel) / pow(numpy.abs(Kernel)**2+(2e-3)**2,0.5) 
            FreqSum+=numpy.fft.rfft(Waveforms[i][j])*Filter/Kernel
            
        SumForm=numpy.fft.irfft(FreqSum)
    print(N)
    AllSums+=SumForm/Energies[i]
    if(PlotThem):
        pylab.figure(figsize=(5,5))    
        pylab.plot(SumForm/Energies[i],label=Count)
        pylab.xlabel("Time")
        pylab.xlim(-10,150)
        pylab.title("Energy = " + str(EnergiesTrue[i]/1000.) + " TeV")
    Count=Count+1
    if Count==Top:
        break
        

